### Imports

In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from pyod.models.hbos import HBOS
from sklearn import cluster

In [2]:
pd.options.mode.chained_assignment = None 

### Datasets Import

Dataset: https://www.kaggle.com/brandao/diabetes?select=diabetic_data.csv

The data set represents 10 years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. 

In [3]:
# Reading Datasets
df_ecg_diabetic = pd.read_csv('diabetic_data.csv')

### Feature Description

1. Patient_Nbr: Patient Identification Number
2. Age: Patient Age
3. Gender: Patient Gender
4. Time_In_Hospital: No. of days patient was admitted for
5. Num_Medications: No. of medications patient was on
6. Num_Lab_Procedures: No. of lab procedures performed on patient
7. State: Patient US State
8. Hospital_Id: Hospital he was admitted to
9. Disease: Patient's Disease
10. Smoker: If patient was smoker
11. Prevalent_Stroke: whether or not the patient had previously had a stroke
12. Prevalent_Hyp: whether or not the patient was hypertensive 
13. Diabetes: whether or not the patient had diabetes 
14. Cholestrol: total cholesterol level 
15. SysBP: systolic blood pressure 
16. DiaBP: diastolic blood pressure 
17. BMI: Body Mass Index
18. Heart_Rate: heart rate
19. Glucose: glucose level 
20. Diet_Followed: whether or not patient followed his diet chart
21. Med_Followed: whether or not patient followed his medicine routine chart
22. Steps: Number of steps walked by patient
23. Calories_Burnt: Number of calories burnt by patient

### Data Engineering

In [4]:
df_ecg_diabetic.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [5]:
# Taking partial required data from ecg_diabetic
overall_dataset = df_ecg_diabetic[['patient_nbr', 'age', 'gender', 'time_in_hospital', 
                                   'num_medications', 'num_lab_procedures']]

In [6]:
# Creating new columns
overall_dataset[['State', 'Hospital_Id', 'Disease', 'Smoker', 'Prevalent_Stroke', 'Prevalent_Hyp', 'Diabetes', 'Cholestrol', 'SysBP', 'DiaBP', 'BMI', 'Heart_Rate', 'Glucose', 'Diet_Followed', 'Med_Followed', 'Steps', 'Calories_Burnt']] = None

In [7]:
# Adding states in dataframe
state_list = ['California', 'Texas', 'Washington', 'Virginia', 'Florida']
overall_dataset['State'] = np.random.choice(state_list, df_ecg_diabetic.shape[0], p= [0.23,0.25,0.2,0.12,0.2])

# Adding diseases in dataframe
disease_list = ['Coronary Artery Bypass Grafting (CABG)', 'Heart Transplant', 'Bypass Surgery', 'COPD (Chronic Obstructive Pulmonary Disease)', 'Lung Cancer', 'Urinary Tract Infection', 'Kidney Stones', 'Melanoma', 'Knee Arthroplasty']
overall_dataset['Disease'] = np.random.choice(disease_list, df_ecg_diabetic.shape[0], p=[0.05,0.07,0.1,0.06,0.11,0.2,0.2,0.03,0.18])

# Adding hospital id in dataframe
hospitals = ['HCA Healthcare', 'Veteran\'s Health Administration', 'Ascension', 'Common Spirit Health']
overall_dataset['Hospital_Id'] = np.random.choice(hospitals, df_ecg_diabetic.shape[0])

In [8]:
# Changing age to number

for i in range(overall_dataset.shape[0]):
    if overall_dataset.loc[i, 'age'] == '[0-10)': 
        overall_dataset.at[i, 'age'] = np.random.randint(1, 10)
    elif overall_dataset.loc[i, 'age'] == '[10-20)': 
        overall_dataset.at[i, 'age'] = np.random.randint(11, 20)
    elif overall_dataset.loc[i, 'age'] == '[20-30)': 
        overall_dataset.at[i, 'age'] = np.random.randint(21, 30)
    elif overall_dataset.loc[i, 'age'] == '[30-40)': 
        overall_dataset.at[i, 'age'] = np.random.randint(31, 40)
    elif overall_dataset.loc[i, 'age'] == '[40-50)': 
        overall_dataset.at[i, 'age'] = np.random.randint(41, 50)
    elif overall_dataset.loc[i, 'age'] == '[50-60)': 
        overall_dataset.at[i, 'age'] = np.random.randint(51, 60)
    elif overall_dataset.loc[i, 'age'] == '[60-70)': 
        overall_dataset.at[i, 'age'] = np.random.randint(61, 70)
    elif overall_dataset.loc[i, 'age'] == '[70-80)': 
        overall_dataset.at[i, 'age'] = np.random.randint(71, 80)
    elif overall_dataset.loc[i, 'age'] == '[80-90)': 
        overall_dataset.at[i, 'age'] = np.random.randint(81, 90)
    elif overall_dataset.loc[i, 'age'] == '[90-100)': 
        overall_dataset.at[i, 'age'] = np.random.randint(91, 100)



In [9]:
# Adding Smoker tab
count = overall_dataset.loc[overall_dataset['age']>15].shape[0]
overall_dataset.loc[overall_dataset['age']>15,['Smoker']] = np.random.choice([0,1], count, p=[0.75,0.25]) # 1 is for smoker
overall_dataset.loc[overall_dataset['Smoker'].isna(),'Smoker'] = 0

In [10]:
# Prevalent Stroke
count = overall_dataset[((overall_dataset['Disease'] == 'Coronary Artery Bypass Grafting (CABG)') |   
                (overall_dataset['Disease'] == 'Heart Transplant') |
                (overall_dataset['Disease'] == 'Bypass Surgery')) & (overall_dataset['age']>10)].shape[0]

overall_dataset.loc[((overall_dataset['Disease'] == 'Coronary Artery Bypass Grafting (CABG)') |   
                (overall_dataset['Disease'] == 'Heart Transplant') |
                (overall_dataset['Disease'] == 'Bypass Surgery')) & (overall_dataset['age']>10),['Prevalent_Stroke']] = np.random.choice([0,1], count, p=[0.35,0.65]) # 1 is for smoker

overall_dataset.loc[overall_dataset['Prevalent_Stroke'].isna(),'Prevalent_Stroke'] = 0


In [11]:
# Prevalent Hypertension
count = overall_dataset[overall_dataset['age']>20].shape[0]
overall_dataset.loc[overall_dataset['age']>20, 'Prevalent_Hyp'] = np.random.choice([0,1], count, p=[0.75, 0.25])
overall_dataset.loc[overall_dataset['Prevalent_Hyp'].isna(),'Prevalent_Hyp'] = 0


In [12]:
# Diabetes

count_15 = overall_dataset[overall_dataset['age']<=15].shape[0]
overall_dataset.loc[overall_dataset['age']<=15, 'Diabetes'] = np.random.choice([0,1], count_15, p=[0.98, 0.02])

count_30 = overall_dataset[(overall_dataset['age']>15) & (overall_dataset['age']<=30)].shape[0]
overall_dataset.loc[(overall_dataset['age']>15) & (overall_dataset['age']<=30), 'Diabetes'] = np.random.choice([0,1], count_30, p=[0.8, 0.2])


count_50 = overall_dataset[(overall_dataset['age']>30) & (overall_dataset['age']<=50)].shape[0]
overall_dataset.loc[(overall_dataset['age']>30) & (overall_dataset['age']<=50), 'Diabetes'] = np.random.choice([0,1], count_50, p=[0.55, 0.45])

count_100 = overall_dataset[overall_dataset['age']>50].shape[0]
overall_dataset.loc[overall_dataset['age']>50, 'Diabetes'] = np.random.choice([0,1], count_100, p=[0.25, 0.75])

In [13]:
# BMI
overall_dataset['BMI'] = np.random.uniform(10, 60, overall_dataset.shape[0])

In [14]:
# Cholestrol
for i in range(overall_dataset.shape[0]):
    if overall_dataset.loc[i, 'BMI'] <= 20:
        overall_dataset.loc[i, 'Cholestrol'] = np.random.uniform(low = 150, high = 200)
    elif overall_dataset.loc[i, 'BMI'] > 20 and overall_dataset.loc[i, 'BMI'] <= 40:
        overall_dataset.loc[i, 'Cholestrol'] = np.random.uniform(low = 160, high = 240)
    elif overall_dataset.loc[i, 'BMI'] > 40 and overall_dataset.loc[i, 'BMI'] <= 60:
        overall_dataset.loc[i, 'Cholestrol'] = np.random.uniform(low = 170, high = 250)

In [15]:
# SysBP/ DiaBP
for i in range(overall_dataset.shape[0]):
    if overall_dataset.loc[i, 'Prevalent_Hyp'] == 1:
        overall_dataset.loc[i, 'SysBP'] = np.random.uniform(low = 95, high = 170)
        overall_dataset.loc[i, 'DiaBP'] = np.random.uniform(low = 60, high = 140)
    else:
        overall_dataset.loc[i, 'SysBP'] = np.random.uniform(low = 110, high = 160)
        overall_dataset.loc[i, 'DiaBP'] = np.random.uniform(low = 70, high = 130)

In [16]:
# Heart Rate
overall_dataset['Heart_Rate'] = np.random.randint(low = 50, high = 100, size = overall_dataset.shape[0])

In [17]:
# Glucose 
for i in range(overall_dataset.shape[0]):
    if overall_dataset.loc[i, 'Diabetes'] == 0:
        overall_dataset.loc[i, 'Glucose'] = np.random.uniform(low = 60, high = 140)
    else:
        overall_dataset.loc[i, 'Glucose'] = np.random.uniform(low = 110, high = 250)


In [18]:
# Gender String To Numeric
overall_dataset['gender'] = np.select([overall_dataset['gender']=='Female', overall_dataset['gender']=='Male'],
                                     [1,0], default = np.NaN)

In [19]:
# Steps
overall_dataset['Steps'] = np.random.randint(low = 1500, high = 15000, size = overall_dataset.shape[0])

In [20]:
# Calories
for i in range(overall_dataset.shape[0]):
    if overall_dataset.loc[i, 'Steps'] <= 5000:
        overall_dataset.loc[i, 'Calories_Burnt'] = np.random.randint(low = 65, high = 270)
    elif overall_dataset.loc[i, 'Steps'] > 5000 and overall_dataset.loc[i, 'Steps'] <= 10000:
        overall_dataset.loc[i, 'Calories_Burnt'] = np.random.randint(low = 200, high = 500)
    else:
        overall_dataset.loc[i, 'Calories_Burnt'] = np.random.randint(low = 400, high = 1000)

In [21]:
# Diet_Followed
overall_dataset['Diet_Followed'] = np.random.choice([0,1], overall_dataset.shape[0], p = [0.65, 0.35])

In [22]:
# Med_Followed
overall_dataset['Med_Followed'] = np.random.choice([0,1], overall_dataset.shape[0], p = [0.7, 0.3])

In [23]:
overall_dataset.head()

,patient_nbr,age,gender,time_in_hospital,num_medications,num_lab_procedures,State,Hospital_Id,Disease,Smoker,...,Cholestrol,SysBP,DiaBP,BMI,Heart_Rate,Glucose,Diet_Followed,Med_Followed,Steps,Calories_Burnt
0,8222157,1,1.0,1,1,41,California,Ascension,Urinary Tract Infection,0,...,207.98,132.107,80.7909,55.433178,94,77.3306,0,0,6090,263
1,55629189,11,1.0,3,18,59,Texas,HCA Healthcare,Knee Arthroplasty,0,...,220.765,134.828,70.5023,39.361805,90,119.278,0,0,8340,443
2,86047875,25,1.0,2,13,11,California,Common Spirit Health,COPD (Chronic Obstructive Pulmonary Disease),0,...,231.726,121.056,74.699,29.317731,51,135.454,0,0,13885,533
3,82442376,34,0.0,2,16,44,California,Ascension,Kidney Stones,0,...,249.924,115.229,108.614,43.007455,64,155.511,0,0,7901,495
4,42519267,42,0.0,1,8,51,California,Veteran's Health Administration,Coronary Artery Bypass Grafting (CABG),0,...,193.928,157.714,90.9233,53.554976,51,163.077,1,1,7847,315


In [24]:
# Renaming and Arranging Column 
overall_dataset.columns = ['Patient_Nbr', 'Age', 'Gender', 'Time_In_Hospital', 'Num_Medications',
                                   'Num_Lab_Procedures', 'State', 'Hospital_Id',
                                   'Disease', 'Smoker', 'Prevalent_Stroke', 'Prevalent_Hyp', 'Diabetes',
                                   'Cholestrol', 'SysBP', 'DiaBP', 'BMI', 'Heart_Rate', 'Glucose', 'Diet_Followed',
                                   'Med_Followed', 'Calories_Burnt', 'Steps']

overall_dataset = overall_dataset[['Patient_Nbr', 'Age', 'Gender', 'Time_In_Hospital', 'Num_Medications',
                                   'Num_Lab_Procedures', 'State', 'Hospital_Id',
                                   'Disease', 'Smoker', 'Prevalent_Stroke', 'Prevalent_Hyp', 'Diabetes',
                                   'Cholestrol', 'SysBP', 'DiaBP', 'BMI', 'Heart_Rate', 'Glucose', 'Diet_Followed',
                                   'Med_Followed', 'Steps', 'Calories_Burnt']]

In [25]:
# Dropping Null Value
overall_dataset.dropna(inplace=True) # 3 Null values 
overall_dataset.isnull().sum()

Patient_Nbr           0
Age                   0
Gender                0
Time_In_Hospital      0
Num_Medications       0
Num_Lab_Procedures    0
State                 0
Hospital_Id           0
Disease               0
Smoker                0
Prevalent_Stroke      0
Prevalent_Hyp         0
Diabetes              0
Cholestrol            0
SysBP                 0
DiaBP                 0
BMI                   0
Heart_Rate            0
Glucose               0
Diet_Followed         0
Med_Followed          0
Steps                 0
Calories_Burnt        0
dtype: int64

In [27]:
import sqlite3

In [28]:
con = sqlite3.connect('healthcare.db')

In [29]:
cur = con.cursor()

# Create Table
cur.execute('''CREATE TABLE HEALTHCARE
                (Patient_Nbr, Age, Gender, Time_In_Hospital, Num_Medications,
       Num_Lab_Procedures, State, Hospital_Id, Disease, Smoker,
       Prevalent_Stroke, Prevalent_Hyp, Diabetes, Cholestrol, SysBP,
       DiaBP, BMI, Heart_Rate, Glucose, Diet_Followed,
       Med_Followed, Steps, Calories_Burnt)''')

In [30]:
# Inserting into database
overall_dataset.to_sql('HEALTHCARE', con, if_exists='append', index = False)

In [31]:
cur.execute('''Select * from healthcare limit 10''')

In [32]:
print(cur.fetchall())

[(8222157, 1, 1.0, 1, 1, 41, 'California', 'Ascension', 'Urinary Tract Infection', 0, 0, 0, 0, 207.9799804349803, 132.10664553540605, 80.79091756921666, 55.43317843176812, 94, 77.33058675271391, 0, 0, 263, 6090), (55629189, 11, 1.0, 3, 18, 59, 'Texas', 'HCA Healthcare', 'Knee Arthroplasty', 0, 0, 0, 0, 220.76456159816374, 134.8278882401686, 70.5023301372405, 39.361805492029745, 90, 119.27799600050051, 0, 0, 443, 8340), (86047875, 25, 1.0, 2, 13, 11, 'California', 'Common Spirit Health', 'COPD (Chronic Obstructive Pulmonary Disease)', 0, 0, 0, 0, 231.7260821097948, 121.05631489207539, 74.69896461520395, 29.317730825999046, 51, 135.45387139893984, 0, 0, 533, 13885), (82442376, 34, 0.0, 2, 16, 44, 'California', 'Ascension', 'Kidney Stones', 0, 0, 0, 1, 249.92355096012167, 115.22924444765192, 108.61389038739271, 43.007455353532286, 64, 155.51106691678817, 0, 0, 495, 7901), (42519267, 42, 0.0, 1, 8, 51, 'California', "Veteran's Health Administration", 'Coronary Artery Bypass Grafting (CABG)

In [33]:
con.commit()

In [34]:
con.close()